In the following notebook I will show you the algorithmic approach I used to fill (nearly) every Cabin. This is not a guessing/probabilistic approach, cabins are filled in a structured order based on the passengers Homeplanet and its group (from its passengerID).

Cabins are filled in order based on their number, ie if a passenger is in cabin A/05/P, a passenger in a later group cannot be in A/04/P but they could be in A/01/S, or B/01/P

We are defining the components of the cabin by 
A/01/P
A = cabin deck, can take values 'A','B','C','D','E','F','G','T'
01 = cabin number, can take values 0,1,2...
P  = cabin side, can take values 'P', 'S' (presumably 'Port' and 'Starboard' )

Some assumptions
* If two passengers are in the same group then they are on the same side, Appendix A.1
* If two passengers are in the same group then they are from the same home planet, Appendix A.2
* If two passengers share a last name then they are from the same home planet, Appendix A.3
* Home planets restrict which decks a passenger is on, Appendix A.4
** Passengers with Mars as their home planet are in decks 'D','E' or 'F'
** Passengers with Earth as their home planet are in decks 'E','F' or 'G'
** Passengers with Europa as their home planet are in decks 'A','B','C','D','E','T'
** If a passenger has no bills (RoomService + ShoppingMall + Spa + VRDeck + FoodCourt) and has members in its group in different decks then they are restricted to these decks 
*** Earth :'G'
*** Europa: 'B'
*** Mars: 'E','F'
* Children <= 12 in age have no bills, Appendix A.5



# Feature engineering

In [1]:
import pandas as pd 
from collections import defaultdict # Slightly modified from a regular dictionary


training_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
training_data['Set'] = 'Train'
test_data['Set'] = 'Test'

# The combined dataframe we will be using for the rest of this project
df = pd.concat([training_data,test_data]) 


In [2]:
def column_splits(data_frame):
    data_frame[['Group', 'GroupNumber']] = data_frame['PassengerId'].str.split('_', expand=True)

    data_frame[['CabinDeck', 'CabinNumber', 'CabinSide']]= data_frame['Cabin'].str.split("/", expand = True)
    data_frame.CabinNumber = data_frame.CabinNumber.astype('Int64')
    
    data_frame[['FirstName','LastName']] = data_frame['Name'].str.split(" ",expand = True)

    return data_frame

df = column_splits(df)


In [3]:
df['Bills'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
df.loc[(df['Age'] < 13), 'Bills'] = 0
df.loc[(df['CryoSleep'] == True),'Bills'] = 0 
    

In [4]:
def add_group_size_column(dataframe):
    dataframe['GroupSize'] = dataframe.groupby('Group')['Group'].transform('count')
    return dataframe


df = add_group_size_column(df)


In [5]:

def add_potential_decks_column(dataframe):
    
    potential_decks_by_homeplanet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
    }

    potential_decks_by_homeplanet_no_bills = {
        'Earth':['G'],
        'Europa':['B'],
        'Mars': ['E','F']
    }
    
    def func_potential_decks_apply(row):
        if pd.isna(row.Cabin):
            if row.Bills == 0 and not pd.isna(row.HomePlanet):
                
                # Checking if other members of group are in multiple different cabin decks
                if dataframe[dataframe.Group == row.Group].CabinDeck.dropna().nunique() > 1:
                    return potential_decks_by_homeplanet_no_bills[row.HomePlanet]
            
            # If not then it goes to the standard decks for their homeplanet
            if not pd.isna(row.HomePlanet):
                return potential_decks_by_homeplanet[row.HomePlanet]
            
            else:
                
                # If their homeplanet isn't known then they could be in any cabin deck
                return list(dataframe.CabinDeck.dropna().unique())
            
    dataframe['PotentialDecks'] = dataframe.apply(func_potential_decks_apply,axis = 1)
    return dataframe
    
            

def add_potential_sides_column(dataframe):
    
    def func_potential_sides_apply(row):
        if pd.isna(row.Cabin):
            
            # Checks to see if anyone else in their group has a known cabin side
            group = dataframe[dataframe.Group == row.Group].CabinSide.dropna()
            if group.nunique() > 0:
                return [group.iloc[0]]
            
            # If no one else is in their group or they haven't got a known cabin side then the passenger could be on either side
            return ['P','S']
        
    dataframe['PotentialSides'] = dataframe.apply(func_potential_sides_apply,axis = 1)
    return dataframe

    
df = add_potential_decks_column(df)
df = add_potential_sides_column(df)



In [6]:
def impute_attribute_by_shared_features(dataframe,attribute,shared_feature):
    
    # Iterates through all the rows that have nan for this attribute
    for index, row in dataframe[dataframe[attribute].isna()].iterrows():
        rows_with_shared_features = dataframe[dataframe[shared_feature] == row[shared_feature]].dropna(subset=[attribute])
        
        if not rows_with_shared_features.empty:
            dataframe.loc[index, attribute] = rows_with_shared_features[attribute].iloc[0]

    return dataframe

df = impute_attribute_by_shared_features(df,'HomePlanet','Group')
df = impute_attribute_by_shared_features(df,'HomePlanet','LastName')

In [7]:
df = df.sort_values(by = ['Group','GroupNumber'])
df = df.reset_index(drop = True)


# Imputing

In [8]:
def impute_from_cabin_and_index(dataframe,cabin,index):
    dataframe.loc[index,['Cabin','CabinDeck','CabinNumber','CabinSide']] = [cabin,cabin.split("/")[0],int(cabin.split("/")[1]),cabin.split("/")[2]]
    return dataframe

In [9]:
def passengers_empty_cabin_options(dataframe):
    
    df_passengers_without_cabin = dataframe[dataframe['Cabin'].isna()]
    all_passenger_cabin_options = {}

    for passenger_index, passenger in df_passengers_without_cabin.iterrows():
        all_passenger_cabin_options[passenger_index] = []

        for deck in passenger.PotentialDecks:
            for side in passenger.PotentialSides:
                
                # Filter dataframe for the current deck and side
                df_filtered = dataframe[(dataframe['CabinDeck'] == deck) & (dataframe['CabinSide'] == side)]

                # Split into cabins before and after the current passenger index
                max_cabin_no_before = max(df_filtered.loc[df_filtered.index < passenger_index, 'CabinNumber'].dropna().unique(), default = -1 )
                min_cabin_no_after = min(df_filtered.loc[df_filtered.index > passenger_index, 'CabinNumber'].dropna().unique(), default = -1)

                # If no cabins were found of that deck and side before or after the row
                if max_cabin_no_before == -1 or min_cabin_no_after == -1:
                    continue
                
                # If a cabin number is seen before the row and the next cabin number is more than 1 higher after the row
                # then there is an empty cabin it can potentially fill
                if max_cabin_no_before + 1 < min_cabin_no_after:
                    all_passenger_cabin_options[passenger_index] += [f"{deck}/{i}/{side}" for i in range(max_cabin_no_before + 1, min_cabin_no_after)]

    return all_passenger_cabin_options





# solo group and only one room that fits

In [10]:
def solo_group_one_cabin_option(dataframe):
    
    all_passenger_cabin_options = passengers_empty_cabin_options(dataframe)

    # Iterates through all the passengers that haven't got a Cabin yet and are alone in their group (ie can't share)
    for passenger_index in list(df[(df.Cabin.isna()) & (df.GroupSize == 1)].index):

        # If they have only one free cabin that they could fill
        if len(all_passenger_cabin_options[passenger_index]) == 1:
            matching_cabin = all_passenger_cabin_options[passenger_index][0]
            dataframe = impute_from_cabin_and_index(dataframe,matching_cabin,passenger_index)

    return dataframe


# no free rooms so has to share

In [11]:
def no_suitable_cabin_so_shares(dataframe):
    all_passenger_cabin_options = passengers_empty_cabin_options(dataframe)
    
    for passenger_index,passenger_cabin_options in all_passenger_cabin_options.items():
        
        # If there are no free cabins that the passenger can fill
        if not passenger_cabin_options:
            
            passenger_row = dataframe.loc[passenger_index]
            
            # Finding all other group members cabins and filtering them by whether they are in the same deck that the passenger must be in
            passengers_group_cabins = dataframe[(dataframe['Group'] == passenger_row['Group']) &
                                  (dataframe['CabinDeck'].isin(passenger_row['PotentialDecks']))].Cabin.dropna()
            
            # If there is only one Cabin from their group they could share with
            if passengers_group_cabins.nunique() == 1:
                matching_cabin = passengers_group_cabins.iloc[0]
                dataframe = impute_from_cabin_and_index(dataframe,matching_cabin,passenger_index)
                
    return dataframe
    


# only passenger that can take that cabin

In [12]:
def only_matching_passenger_for_cabin(dataframe):
    all_passenger_cabin_options = passengers_empty_cabin_options(dataframe)
    
    cabins_to_fill = defaultdict(list)
    
    # Iterate over cabins to see which passengers can fit that cabin
    for passenger_index, cabin_options in all_passenger_cabin_options.items():
        for cabin in cabin_options:
            cabins_to_fill[cabin].append(passenger_index)
    
    # Iterate over cabin and impute passengers where only one fits
    for cabin, passengers_indices in cabins_to_fill.items():
        if len(passengers_indices) == 1:
            dataframe = impute_from_cabin_and_index(dataframe, cabin, passengers_indices[0])
    
    return dataframe


# all imputes

In [13]:
def all_imputes(dataframe):
    dataframe = solo_group_one_cabin_option(dataframe)
    dataframe = no_suitable_cabin_so_shares(dataframe)
    dataframe = only_matching_passenger_for_cabin(dataframe)

    dataframe = solo_group_one_cabin_option(dataframe)
    dataframe = no_suitable_cabin_so_shares(dataframe)
    dataframe = only_matching_passenger_for_cabin(dataframe)
    
    return dataframe
    
df = all_imputes(df)
df.isna().sum()

B/0/S
B/5/P
F/47/S
G/45/P
B/12/P
B/14/S
B/18/S
C/27/S
A/7/S
A/4/P
C/29/S
D/35/S
E/74/S
B/45/S
C/42/P
E/94/S
B/60/S
G/222/P
C/53/P
G/226/S
C/58/S
F/312/P
F/310/S
G/272/P
G/262/S
C/71/S
D/59/P
F/428/P
G/367/P
G/383/S
G/396/P
A/20/P
D/102/S
G/505/S
B/121/S
G/578/P
B/135/S
F/668/S
G/595/P
G/606/S
F/704/S
G/612/P
G/657/P
G/661/S
C/133/P
C/155/S
C/139/P
B/183/S
C/148/P
B/156/P
C/189/S
E/311/S
F/911/S
A/44/P
F/1025/P
G/832/S
C/201/S
B/202/S
G/857/P
G/904/S
F/1194/P
B/197/P
A/58/P
F/1174/S
C/190/P
G/1015/P
A/63/P
G/1026/P
F/1326/P
G/1039/S
B/249/S
C/197/P
C/206/P
B/219/P
E/450/S
D/214/S
C/235/P
C/280/S
B/301/S
F/1501/S
G/1273/S
G/1274/S
G/1277/P
E/521/S
G/1300/S
E/534/S
G/1313/P
G/1317/P
C/312/S
G/1364/P
G/1394/P
C/324/S
G/1394/S
F/1808/P
B/339/S
F/1757/S
G/1476/P
G/1501/P
B/216/S


PassengerId           0
HomePlanet           12
CryoSleep           310
Cabin                41
Destination         274
Age                 270
VIP                 296
RoomService         263
FoodCourt           289
ShoppingMall        306
Spa                 284
VRDeck              268
Name                294
Transported        4277
Set                   0
Group                 0
GroupNumber           0
CabinDeck            41
CabinNumber          41
CabinSide            41
FirstName           294
LastName            294
Bills               785
GroupSize             0
PotentialDecks    12671
PotentialSides    12671
dtype: int64

In [14]:
df[df.Group == '7442']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,GroupNumber,CabinDeck,CabinNumber,CabinSide,FirstName,LastName,Bills,GroupSize,PotentialDecks,PotentialSides
10410,7442_01,Earth,False,E/495/S,TRAPPIST-1e,43.0,False,20.0,0.0,726.0,...,01,E,495,S,Antony,Morrencis,NaN,2,None,None
10411,7442_02,Earth,True,NaN,PSO J318.5-22,17.0,False,0.0,0.0,0.0,...,02,NaN,<NA>,NaN,Franda,Morrencis,0.0,2,"[E, F, G]",[S]


Earth :'G'
*** Europa: 'B'
*** Mars: 'E','F'

In [15]:
df[(df.Cabin.isna()) & (df.GroupSize > 1) & (df.Bills == 0)]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,GroupNumber,CabinDeck,CabinNumber,CabinSide,FirstName,LastName,Bills,GroupSize,PotentialDecks,PotentialSides
9267,6612_05,Earth,False,NaN,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,...,05,NaN,<NA>,NaN,Maurie,Dickley,0.0,6,"[E, F, G]",[S]
10411,7442_02,Earth,True,NaN,PSO J318.5-22,17.0,False,0.0,0.0,0.0,...,02,NaN,<NA>,NaN,Franda,Morrencis,0.0,2,"[E, F, G]",[S]
12668,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,...,03,NaN,<NA>,NaN,Bath,Brakeng,0.0,5,"[A, B, C, D, E, T]",[P]
12892,9223_01,Mars,True,NaN,TRAPPIST-1e,24.0,False,0.0,0.0,NaN,...,01,NaN,<NA>,NaN,Weessh,Sun,0.0,2,"[D, E, F]","[P, S]"
12893,9223_02,Mars,True,NaN,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,...,02,NaN,<NA>,NaN,Perit,Sun,0.0,2,"[D, E, F]","[P, S]"


In [16]:
df[df.Cabin.isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,GroupNumber,CabinDeck,CabinNumber,CabinSide,FirstName,LastName,Bills,GroupSize,PotentialDecks,PotentialSides
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,...,01,NaN,<NA>,NaN,Tauxon,Suptibler,0.0,1,"[A, B, C, D, E, T]","[P, S]"
421,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,NaN,230.0,0.0,...,01,NaN,<NA>,NaN,Naviton,Coudered,NaN,1,"[A, B, C, D, E, T]","[P, S]"
479,0348_02,Mars,NaN,NaN,TRAPPIST-1e,36.0,False,520.0,0.0,1865.0,...,02,NaN,<NA>,NaN,Weet,Mane,2385.0,2,"[D, E, F]",[P]
505,0364_02,Mars,False,NaN,TRAPPIST-1e,37.0,False,731.0,0.0,517.0,...,02,NaN,<NA>,NaN,Anakes,Chité,1298.0,2,"[D, E, F]",[P]
517,0374_02,Earth,False,NaN,TRAPPIST-1e,36.0,False,6.0,0.0,0.0,...,02,NaN,<NA>,NaN,Tamie,Sterreray,789.0,2,"[E, F, G]",[P]
1429,1011_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,1633.0,4.0,49.0,...,01,NaN,<NA>,NaN,Camie,Prinson,1692.0,2,"[E, F, G]",[P]
1466,1041_01,Europa,True,NaN,TRAPPIST-1e,46.0,False,0.0,0.0,0.0,...,01,NaN,<NA>,NaN,Algrafi,Heedry,0.0,1,"[A, B, C, D, E, T]","[P, S]"
1543,1095_01,Europa,True,NaN,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,...,01,NaN,<NA>,NaN,Alhail,Drelcate,0.0,1,"[A, B, C, D, E, T]","[P, S]"
2442,1709_03,Mars,False,NaN,TRAPPIST-1e,35.0,False,1313.0,0.0,24.0,...,03,NaN,<NA>,NaN,Bleark,Minen,1338.0,7,"[D, E, F]",[S]
2970,2092_03,Mars,False,NaN,TRAPPIST-1e,13.0,NaN,6726.0,0.0,1605.0,...,03,NaN,<NA>,NaN,Cray,Stpie,9597.0,5,"[D, E, F]",[S]


# Manual workings


In [17]:
def all_cabin_options_for_each_row(dataframe):
    all_passenger_cabin_options = passengers_empty_cabin_options(dataframe)
    for passenger_index, passenger_options in all_passenger_cabin_options.items():
        print()
        print("Index:",passenger_index, "GroupSize:", dataframe.iloc[passenger_index].GroupSize)
        print("Free cabins that match:")
        print(passenger_options)
                
             


In [18]:
all_cabin_options_for_each_row(df)


Index: 404 GroupSize: 1
Free cabins that match:
['B/13/P', 'C/13/S']

Index: 421 GroupSize: 1
Free cabins that match:
['B/13/P', 'C/13/S']

Index: 479 GroupSize: 2
Free cabins that match:
['E/20/P', 'E/21/P']

Index: 505 GroupSize: 2
Free cabins that match:
['E/20/P', 'E/21/P']

Index: 517 GroupSize: 2
Free cabins that match:
['E/20/P', 'E/21/P']

Index: 1429 GroupSize: 2
Free cabins that match:
['E/58/P']

Index: 1466 GroupSize: 1
Free cabins that match:
['C/40/S', 'D/36/S', 'E/58/P']

Index: 1543 GroupSize: 1
Free cabins that match:
['C/40/S', 'D/36/S']

Index: 2442 GroupSize: 7
Free cabins that match:
[]

Index: 2970 GroupSize: 5
Free cabins that match:
[]

Index: 3529 GroupSize: 1
Free cabins that match:
['E/150/P', 'F/519/P']

Index: 3530 GroupSize: 1
Free cabins that match:
['E/150/P', 'F/519/P']

Index: 4233 GroupSize: 1
Free cabins that match:
['B/98/P', 'B/99/P']

Index: 4254 GroupSize: 1
Free cabins that match:
['B/98/P', 'B/99/P']

Index: 4569 GroupSize: 3
Free cabins that 

In [19]:




cabin_list = [(1429,'E/58/P'),(8413,'A/57/P'),(9265,'F/1267/S'),(9267,'F/1267/S'),(4233,'B/98/P'),(4254,'B/99/P'),(6493,'E/300/S'),(6514,'E/301/S')]

for index,cabin in cabin_list:
    impute_from_cabin_and_index(df,cabin,index)

"""
df.loc[12892,'Cabin'] = 'F/1785/S' # maybe only one is from this room and the other is joined in the other room
df.loc[12893,'Cabin'] = 'F/1785/S'
df.loc[9267,'Cabin] =  G/1077/S # has no bills so would have to be in G if the other is in F, but maybe could be in F if the other is in G
df.loc[9265,'Cabin'] = F/1267/S  

"""


"\ndf.loc[12892,'Cabin'] = 'F/1785/S' # maybe only one is from this room and the other is joined in the other room\ndf.loc[12893,'Cabin'] = 'F/1785/S'\ndf.loc[9267,'Cabin] =  G/1077/S # has no bills so would have to be in G if the other is in F, but maybe could be in F if the other is in G\ndf.loc[9265,'Cabin'] = F/1267/S  \n\n"

# End


In [20]:
traindata = df[df.Set == 'Train']
testdata = df[df.Set == 'Test']

In [21]:
df_to_comp = pd.read_csv('data/31remaining.csv')
df_to_comp = df_to_comp.rename(columns = {'Number':'CabinNumber'})
df_to_comp['CabinNumber'] = df_to_comp['CabinNumber'].astype('Int64')


In [22]:
for index,row in df.iterrows():
    if not (pd.isna(row.Cabin) and pd.isna(df_to_comp.iloc[index].Cabin)):
        if row.Cabin != df_to_comp.iloc[index].Cabin:
            print(index,row.Cabin, df_to_comp.iloc[index].Cabin)

12892 nan F/1785/S
12893 nan F/1785/S


In [23]:
for index,row in df.iterrows():
    if row.Bills == 0:
        if row.GroupSize > 1:
            if df[df.Group == row.Group].CabinDeck.dropna().nunique() > 1:
                print(row.Group)

0006
0017
0064
0064
0067
0091
0094
0099
0103
0103
0133
0138
0140
0202
0220
0220
0220
0220
0220
0222
0222
0273
0283
0313
0313
0319
0339
0339
0342
0407
0448
0461
0489
0489
0499
0499
0503
0549
0560
0560
0584
0585
0585
0594
0602
0602
0602
0626
0626
0632
0652
0652
0652
0652
0666
0686
0691
0699
0720
0720
0742
0774
0806
0823
0838
0858
0864
0864
0899
0923
0930
0938
0984
0984
0984
0984
0994
1011
1024
1024
1064
1086
1086
1086
1086
1100
1111
1111
1116
1116
1125
1130
1144
1145
1145
1155
1182
1186
1186
1186
1186
1187
1187
1207
1207
1207
1211
1211
1211
1211
1211
1211
1221
1224
1228
1238
1262
1311
1321
1321
1350
1350
1354
1354
1354
1354
1358
1361
1369
1375
1384
1409
1425
1425
1460
1491
1495
1500
1500
1500
1500
1500
1554
1568
1582
1584
1603
1603
1615
1615
1630
1630
1688
1688
1709
1727
1738
1764
1764
1782
1811
1844
1844
1855
1862
1874
1889
1962
1962
2005
2006
2006
2017
2024
2035
2056
2056
2057
2057
2057
2101
2104
2133
2143
2234
2234
2234
2234
2240
2240
2257
2276
2291
2338
2347
2347
2347
2347
2347
2350


# Appendix